
# Stock Forecast (Bootcamp Mini Proje)

Bu notebook, seçtiğiniz bir hissenin **bir sonraki gün (t+1) kapanış fiyatını** basit ve anlaşılır makine öğrenmesi yöntemleriyle tahmin eder.

**İçerik:**
- Veri: Yahoo Finance (`yfinance`)
- Özellikler: `lag1/5/10`, `sma5/10`, `vol5`, `rsi14`
- Modeller: Basit *Naive* (lag1), **Linear Regression**, **Random Forest**
- Metrik: **MAE** (+ R²)
- Grafik: Gerçek vs Tahmin (son 200 gün)
- Son hücre: **Yarın için tahmin**

> **Not:** Bu eğitim amaçlı bir demodur; yatırım tavsiyesi değildir.


In [ ]:

import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

# ---- Parametreler ----
TICKER = "AAPL"         # Örn: "TSLA" veya BIST için "THYAO.IS", "GARAN.IS"
START  = "2018-01-01"   # Veri başlangıç tarihi
END    = None           # None = bugüne kadar
PREDICT_HORIZON = 1     # t+1 tahmini; 5 yaparsanız t+5 olur.


In [ ]:

raw = yf.download(TICKER, start=START, end=END, auto_adjust=True)
raw = raw.dropna()
display(raw.tail())
print(f"Toplam gözlem: {len(raw)}")


In [ ]:

df = raw.copy()
df["ret"]   = df["Close"].pct_change()
df["lag1"]  = df["Close"].shift(1)
df["lag5"]  = df["Close"].shift(5)
df["lag10"] = df["Close"].shift(10)
df["sma5"]  = df["Close"].rolling(5).mean()
df["sma10"] = df["Close"].rolling(10).mean()
df["vol5"]  = df["ret"].rolling(5).std()

# RSI (14)
delta = df["Close"].diff()
gain = delta.clip(lower=0).rolling(14).mean()
loss = -delta.clip(upper=0).rolling(14).mean()
rs = gain / (loss + 1e-9)
df["rsi14"] = 100 - (100 / (1 + rs))

# Hedef (y): t+1 kapanış
df["y"] = df["Close"].shift(-PREDICT_HORIZON)

features = ["lag1","lag5","lag10","sma5","sma10","vol5","rsi14"]
data = df.dropna().copy()
X, y = data[features], data["y"]
display(data[[*features, "y"]].tail())
print("Özellik şekli:", X.shape, "| Hedef uzunluğu:", len(y))


In [ ]:

# Zaman bazlı (shuffle yok) 80/20 bölme
split = int(len(X) * 0.8)
X_train, y_train = X.iloc[:split], y.iloc[:split]
X_test,  y_test  = X.iloc[split:], y.iloc[split:]

# Naive baseline: yhat = bugünkü kapanış (lag1)
yhat_naive = X_test["lag1"].values
mae_naive = mean_absolute_error(y_test, yhat_naive)
print("Naive MAE:", round(mae_naive, 4))


In [ ]:

lr = LinearRegression()
lr.fit(X_train, y_train)
yhat_lr = lr.predict(X_test)

mae_lr = mean_absolute_error(y_test, yhat_lr)
r2_lr  = r2_score(y_test, yhat_lr)
print("LR  - MAE:", round(mae_lr, 4), "| R2:", round(r2_lr, 4))


In [ ]:

rf = RandomForestRegressor(
    n_estimators=300,
    max_depth=8,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)
yhat_rf = rf.predict(X_test)

mae_rf = mean_absolute_error(y_test, yhat_rf)
r2_rf  = r2_score(y_test, yhat_rf)
print("RF  - MAE:", round(mae_rf, 4), "| R2:", round(r2_rf, 4))


In [ ]:

res = pd.DataFrame({
    "Model": ["Naive(lag1)", "LinearRegression", "RandomForest"],
    "MAE":   [mae_naive, mae_lr, mae_rf],
    "R2":    [np.nan, r2_lr, r2_rf]
}).sort_values("MAE")
res


In [ ]:

plot_df = pd.DataFrame({
    "y_true": y_test,
    "yhat_lr": yhat_lr,
    "yhat_rf": yhat_rf
}, index=y_test.index)

plt.figure(figsize=(10,4))
plot_df[["y_true", "yhat_lr", "yhat_rf"]].tail(200).plot(figsize=(12,4))
plt.title(f"{TICKER} – Son 200 gün: Gerçek vs Tahmin")
plt.xlabel("Tarih"); plt.ylabel("Fiyat")
plt.tight_layout()
plt.show()


In [ ]:

last = df.dropna().iloc[-1]
x_pred = pd.DataFrame([{f: last[f] for f in features}])

next_lr = float(lr.predict(x_pred))
next_rf = float(rf.predict(x_pred))

print("Bugünkü Close:", round(last["Close"], 2))
print("Yarın Tahmin (LR):", round(next_lr, 2))
print("Yarın Tahmin (RF):", round(next_rf, 2))
print("\nNot: Tahminleri ±MAE aralığıyla düşünmek daha gerçekçidir.")


In [ ]:

# Opsiyonel: RF özellik önemleri
imp = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
display(imp)



## Notlar
- **Naive**: "Yarın = Bugünkü kapanış" basit kuralıdır; model bundan **daha iyi** olmalı ki değer üretsin.
- **R²** pozitifse model varyansı anlamlı açıklıyor demektir (negatif olabilir, sorun değil; MAE birincil metriktir).
- **Uyarı:** Zaman serilerinde ileri bilgi sızıntısına (data leakage) dikkat. Bu notebook'ta yalnızca geçmiş veriden özellik üretildi.
- **Genişletme fikirleri:**
  - Parametre tarama (GridSearch) ile RF'yi iyileştirme
  - Hedefi getiri (yüzde değişim) olarak modelleyip fiyata geri çevirmek
  - LSTM/GRU gibi derin öğrenme modelleri eklemek (daha uzun süre, daha çok kod)
